# Exploring the Impacts of Architecture and Scale on GNN Performance on Relational Data
By: Joseph Guman, Atindra Jha, and Christopher Pondoc

## Introduction
Welcome back to Relbench! In this tutorial, we'll dive a bit deeper into the benchmark + Relational Deep Learning and explore several choices around architecture, scale, and generalizability. In particular, we'll look to answer the following questions:

1. Can we train our Relational Deep Learning on one entity classification task and expect strong zero-shot performance on another entity classification task? What happens if we finetune the model?
2. How does our choice of using embedding models to generate expressive node features impact our performance on node classification tasks?
3. How can we alter and/or extend the architecture of our existing Relational Deep Learning model to improve performance on different tasks?

This notebook already assumes you've looked through the tutorials on [loading in data](https://github.com/snap-stanford/relbench/blob/main/tutorials/load_data.ipynb) and [training a model](https://github.com/snap-stanford/relbench/blob/main/tutorials/train_model.ipynb), as our walkthrough uses those guides as a launchpad to explore deeper questions. If you haven't had a chance to look through those notebooks, we suggest starting there first.

With all that being said, let's get started!

## Question 1: Can we generalize?
Let's take a look at our first question, which involves looking at whether our Relational Deep Learning model can generalize to other tasks with/without finetuning.

Let's first start by looking setting up Relbench. As with the other tutorials, we're taking a look at the `rel-f1` dataset and focusing on node classification tasks. We'll begin by training a model on the `driver-dnf` task, which predicts whether a driver will not finish a race in the next month.

In [1]:
from src.tasks.tasks import initialize_task, db_to_graph
import torch
from torch.nn import BCEWithLogitsLoss
from torch_geometric.seed import seed_everything

# Set up dataset and task, define metrics and loss
dataset, task, train_table, val_table, test_table = initialize_task(
    "rel-f1", "driver-dnf"
)
loss_fn = BCEWithLogitsLoss()

# Set up device
seed_everything(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/cpondoc/classes/cs224w/project/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We can then preprocess all of our Relbench data.

In [2]:
import os
from relbench.modeling.graph import make_pkey_fkey_graph
from torch_frame.config.text_embedder import TextEmbedderConfig
from src.embeddings.glove import GloveTextEmbedding

# Preprocess the database data and set up our text embedder
db, col_to_stype_dict = db_to_graph(dataset)
text_embedder_cfg = TextEmbedderConfig(
    text_embedder=GloveTextEmbedding(device=device), batch_size=128
)

# Load in data used to train model
root_dir = "./data"
data, col_stats_dict = make_pkey_fkey_graph(
    db,
    col_to_stype_dict=col_to_stype_dict,
    text_embedder_cfg=text_embedder_cfg,
    cache_dir=os.path.join(root_dir, f"rel-f1_materialized_cache"),
)

Loading Database object from /home/cpondoc/.cache/relbench/rel-f1/db...
Done in 0.02 seconds.


/home/cpondoc/classes/cs224w/project/env/lib/python3.9/site-packages/torch_frame/utils/io.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tf_dict, col_stats = torch.loa

Next, let's load in the data and have our model set up.

In [3]:
from src.models.loader import get_loader
from src.models.rdl import RDLModel

# Set up data loader and model
loader_dict, entity_table = get_loader(train_table, val_table, test_table, task, data)
model = RDLModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)

# if you try out different RelBench tasks you will need to change these
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
epochs = 10

Finalize, let's initialize our training run, and evaluate our model!

In [4]:
from src.models.training import eval_model, training_run

# Get model after a training run
state_dict = training_run(
    model, device, optimizer, task, loader_dict, val_table, loss_fn, entity_table
)
model.load_state_dict(state_dict)

# Evaluate on val and test set
eval_model(model, loader_dict, "val", task, device, val_table)
eval_model(model, loader_dict, "test", task, device, None)

100%|██████████| 23/23 [00:02<00:00,  7.94it/s]


Epoch: 01, Train loss: 0.3705480691610633, Val metrics: {'average_precision': np.float64(0.8436613889307779), 'accuracy': 0.7791519434628975, 'f1': np.float64(0.8758689175769613), 'roc_auc': np.float64(0.6120816326530613)}


100%|██████████| 23/23 [00:02<00:00,  8.83it/s]


Epoch: 02, Train loss: 0.34177915121595737, Val metrics: {'average_precision': np.float64(0.8780304661008016), 'accuracy': 0.7720848056537103, 'f1': np.float64(0.8690355329949239), 'roc_auc': np.float64(0.6786938775510204)}


100%|██████████| 23/23 [00:02<00:00,  8.78it/s]


...

Epoch: 08, Train loss: 0.29883380335445586, Val metrics: {'average_precision': np.float64(0.8919457775955992), 'accuracy': 0.7226148409893993, 'f1': np.float64(0.8287895310796074), 'roc_auc': np.float64(0.6818503401360545)}


100%|██████████| 23/23 [00:02<00:00,  8.69it/s]


Epoch: 09, Train loss: 0.29252413555100887, Val metrics: {'average_precision': np.float64(0.8912859481939854), 'accuracy': 0.5848056537102474, 'f1': np.float64(0.6785225718194254), 'roc_auc': np.float64(0.6820680272108843)}


100%|██████████| 23/23 [00:02<00:00,  8.69it/s]


Epoch: 10, Train loss: 0.294855005459994, Val metrics: {'average_precision': np.float64(0.8903698434213752), 'accuracy': 0.7279151943462897, 'f1': np.float64(0.8265765765765766), 'roc_auc': np.float64(0.681814058956916)}
Best val metrics: {'average_precision': np.float64(0.8955699358038636), 'accuracy': 0.7031802120141343, 'f1': np.float64(0.8116591928251121), 'roc_auc': np.float64(0.6891972789115647)}
Best test metrics: {'average_precision': np.float64(0.847632311657143), 'accuracy': 0.7022792022792023, 'f1': np.float64(0.804490177736202), 'roc_auc': np.float64(0.7114087737276142)}


As we can see, we are able to roughly replicate the results from the [core Relbench paper](https://huggingface.co/spaces/relbench/leaderboard). However, do the results generalize? To do so, let's load in the data for the other entity classification task within `rel-f1` -- `driver-top3` -- and see how we do. 

In [5]:
# Reuse functions to set up `driver-top3 task`
dataset, task, train_table, val_table, test_table = initialize_task(
    "rel-f1", "driver-top3"
)
db, col_to_stype_dict = db_to_graph(dataset)
data, col_stats_dict = make_pkey_fkey_graph(
    db,
    col_to_stype_dict=col_to_stype_dict,
    text_embedder_cfg=text_embedder_cfg,
    cache_dir=os.path.join(root_dir, f"rel-f1_materialized_cache"),
)

loader_dict, entity_table = get_loader(train_table, val_table, test_table, task, data)
model = RDLModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)
model.load_state_dict(state_dict)
eval_model(model, loader_dict, "test", task, device, None)

/home/cpondoc/classes/cs224w/project/env/lib/python3.9/site-packages/torch_frame/utils/io.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tf_dict, col_stats = torch.loa

Best test metrics: {'average_precision': np.float64(0.11374549278031856), 'accuracy': 0.23415977961432508, 'f1': np.float64(0.2299168975069252), 'roc_auc': np.float64(0.22875731605351166)}


Unfortunately, trying out our model zero-shot does not yield amazing results. However, what happens if we use this model as a starting point for finetuning on the task? Let's experiment on fine-tuning this model with fewer epochs on the `driver-top3` task and checking its performance.

In [6]:
# Get model after a training run
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
state_dict = training_run(
    model,
    device,
    optimizer,
    task,
    loader_dict,
    val_table,
    loss_fn,
    entity_table,
    epochs=5,
    state_dict=state_dict,
)
model.load_state_dict(state_dict)

# Evaluate on val and test set
eval_model(model, loader_dict, "val", task, device, val_table)
eval_model(model, loader_dict, "test", task, device, None)

100%|██████████| 3/3 [00:00<00:00,  7.07it/s]


Epoch: 01, Train loss: 1.4039089307023787, Val metrics: {'average_precision': np.float64(0.17205826100401703), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.4553761803228754)}


100%|██████████| 3/3 [00:00<00:00,  7.34it/s]


Epoch: 02, Train loss: 0.4765360890849819, Val metrics: {'average_precision': np.float64(0.19425096939544184), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.52905341240974)}


100%|██████████| 3/3 [00:00<00:00,  7.27it/s]


Epoch: 03, Train loss: 0.47430402036076197, Val metrics: {'average_precision': np.float64(0.2301483474600275), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.6136066366845245)}


100%|██████████| 3/3 [00:00<00:00,  7.40it/s]


Epoch: 04, Train loss: 0.4691849634414766, Val metrics: {'average_precision': np.float64(0.2655475905244805), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.6628263245596747)}


100%|██████████| 3/3 [00:00<00:00,  7.33it/s]


Epoch: 05, Train loss: 0.4515120375182835, Val metrics: {'average_precision': np.float64(0.3010512788704124), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.6643672394330865)}
Best val metrics: {'average_precision': np.float64(0.30320109456583333), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.664582250810772)}
Best test metrics: {'average_precision': np.float64(0.2892614892577663), 'accuracy': 0.8236914600550964, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.7408875940635451)}


Nice! It looks like after we finetune even after just one epoch. we're able to practically replicate the Relbench results. Finally, let's compare this approach to simply training on the task from scratch.

In [7]:
# Define a new model, don't load in old weights.
base_model = RDLModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)
base_optimizer = torch.optim.Adam(base_model.parameters(), lr=0.005)
base_state_dict = training_run(
    base_model,
    device,
    base_optimizer,
    task,
    loader_dict,
    val_table,
    loss_fn,
    entity_table,
    epochs=10,
)
base_model.load_state_dict(base_state_dict)

# Evaluate on val and test set
eval_model(base_model, loader_dict, "val", task, device, val_table)
eval_model(base_model, loader_dict, "test", task, device, None)

100%|██████████| 3/3 [00:00<00:00,  7.31it/s]


Epoch: 01, Train loss: 0.5506209522728028, Val metrics: {'average_precision': np.float64(0.22983284006214555), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.602802314955833)}


100%|██████████| 3/3 [00:00<00:00,  7.35it/s]


Epoch: 02, Train loss: 0.4472975869444856, Val metrics: {'average_precision': np.float64(0.2518872188386706), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.6169572306534555)}


100%|██████████| 3/3 [00:00<00:00,  7.36it/s]


...

Epoch: 08, Train loss: 0.3809619241085038, Val metrics: {'average_precision': np.float64(0.3014899796962464), 'accuracy': 0.7976190476190477, 'f1': np.float64(0.0), 'roc_auc': np.float64(0.694594255612693)}


100%|██████████| 3/3 [00:00<00:00,  7.35it/s]


Epoch: 09, Train loss: 0.34680816557613553, Val metrics: {'average_precision': np.float64(0.37179270042489915), 'accuracy': 0.8010204081632653, 'f1': np.float64(0.15827338129496402), 'roc_auc': np.float64(0.7261471752880256)}


100%|██████████| 3/3 [00:00<00:00,  7.45it/s]


Epoch: 10, Train loss: 0.3288967285170347, Val metrics: {'average_precision': np.float64(0.36854971852014695), 'accuracy': 0.7568027210884354, 'f1': np.float64(0.3286384976525822), 'roc_auc': np.float64(0.7390120227195355)}
Best val metrics: {'average_precision': np.float64(0.36828136931783517), 'accuracy': 0.7568027210884354, 'f1': np.float64(0.3286384976525822), 'roc_auc': np.float64(0.7388865994158857)}
Best test metrics: {'average_precision': np.float64(0.410840878349891), 'accuracy': 0.8071625344352618, 'f1': np.float64(0.29292929292929293), 'roc_auc': np.float64(0.7725948474080269)}


Ultimately, we don't see much of a difference from starting from random weights to using a model pre-initialized from another entity classification task.

### Challenge
Does this trend necessarily work on larger and more diverse datasets? Depending on your compute availability, try out using different datasets, like `rel-amazon`, as well as across different types of tasks!

## Question 2: Different expressiveness of node features?
Next, let's take a look at using different embedding models for node features.

The embedding models are used to help turn the tabular data into usable node features. In the Relbench tutorial, the team uses GloVe embeddings, but the paper also mentions utilizing BERT-style embeddings. In traditional NLP, BERT embeddings are much more popular given that they are contextual -- the vector representation depends on the surrounding words, compared to static embeddings used by GloVe -- and can handle words outside of their vocabulary. In addition, their embedding size is $768$ compared to GloVe's $300$, which introduces an opportunity for more expressiveness.

As an investigation, let's switch out our GloVe embedding model with BERT and retrain a new model from scratch on the `driver-dnf` task.

In [8]:
from src.embeddings.bert import BertTextEmbedding

dataset, task, train_table, val_table, test_table = initialize_task(
    "rel-f1", "driver-dnf"
)

# Preprocess the database data and set up our text embedder
db, col_to_stype_dict = db_to_graph(dataset)
text_embedder_cfg = TextEmbedderConfig(
    text_embedder=BertTextEmbedding(device=device), batch_size=128
)

# Load in data used to train model
data, col_stats_dict = make_pkey_fkey_graph(
    db,
    col_to_stype_dict=col_to_stype_dict,
    text_embedder_cfg=text_embedder_cfg,
    cache_dir=os.path.join(root_dir, f"rel-f1_materialized_cache"),
)
loader_dict, entity_table = get_loader(train_table, val_table, test_table, task, data)

# Initialize new, untrained model using BERT embeddings
bert_model = RDLModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)
bert_optimizer = torch.optim.Adam(bert_model.parameters(), lr=0.005)
bert_state_dict = training_run(
    bert_model,
    device,
    bert_optimizer,
    task,
    loader_dict,
    val_table,
    loss_fn,
    entity_table,
    epochs=10,
)
bert_model.load_state_dict(bert_state_dict)

# Evaluate on val and test set
eval_model(bert_model, loader_dict, "val", task, device, val_table)
eval_model(bert_model, loader_dict, "test", task, device, None)

/home/cpondoc/classes/cs224w/project/env/lib/python3.9/site-packages/torch_frame/utils/io.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tf_dict, col_stats = torch.loa

Epoch: 01, Train loss: 0.37271471863645017, Val metrics: {'average_precision': np.float64(0.8473152771511769), 'accuracy': 0.7791519434628975, 'f1': np.float64(0.8758689175769613), 'roc_auc': np.float64(0.6238367346938776)}


100%|██████████| 23/23 [00:02<00:00,  8.66it/s]


Epoch: 02, Train loss: 0.3378057693444756, Val metrics: {'average_precision': np.float64(0.8805192265018585), 'accuracy': 0.6395759717314488, 'f1': np.float64(0.743073047858942), 'roc_auc': np.float64(0.6601904761904763)}


100%|██████████| 23/23 [00:02<00:00,  8.27it/s]


...

Epoch: 08, Train loss: 0.2879258215730358, Val metrics: {'average_precision': np.float64(0.8966036278446728), 'accuracy': 0.7862190812720848, 'f1': np.float64(0.8727655099894848), 'roc_auc': np.float64(0.700952380952381)}


100%|██████████| 23/23 [00:02<00:00,  8.47it/s]


Epoch: 09, Train loss: 0.2932957138723521, Val metrics: {'average_precision': np.float64(0.8976640284772309), 'accuracy': 0.5936395759717314, 'f1': np.float64(0.6965699208443272), 'roc_auc': np.float64(0.7031655328798185)}


100%|██████████| 23/23 [00:02<00:00,  8.51it/s]


Epoch: 10, Train loss: 0.28281806396953313, Val metrics: {'average_precision': np.float64(0.8995612466446161), 'accuracy': 0.7067137809187279, 'f1': np.float64(0.8028503562945368), 'roc_auc': np.float64(0.7085532879818592)}
Best val metrics: {'average_precision': np.float64(0.8994326716430203), 'accuracy': 0.7067137809187279, 'f1': np.float64(0.8028503562945368), 'roc_auc': np.float64(0.7081179138321995)}
Best test metrics: {'average_precision': np.float64(0.8359322021592208), 'accuracy': 0.717948717948718, 'f1': np.float64(0.7924528301886793), 'roc_auc': np.float64(0.7259454447860244)}


We ultimately don't see that drastic of a difference between using BERT embeddings and GloVe embeddings. Despite being trained differently, the fact that the models are close in size and perform similarly on [general embedding benchmarks](https://huggingface.co/spaces/mteb/leaderboard) may suggest that the results will not be that drastic. 

### Challenge
We encourage you to try larger models with even larger embedding dimensions -- to do so, use our `CustomTextEmbedding` class! To use this class, import it as below, and then specify the name of a model as used on HuggingFace:

```python
from src.embeddings.custom import CustomTextEmbedding
text_embedder_cfg = TextEmbedderConfig(
    text_embedder=CustomTextEmbedding(model_name=<INSERT_HUGGINGFACE_MODEL_HERE>, device=device), batch_size=128
)
```

## Question 3: Different RDL model architectures?
Finally, we experiment with different RDL model architectures. In particular, we investigate what happens as we add or subtract GNN layers from our model.

First, we double the number of GNN layers in our RDL pipeline, moving from `num_layers=2` to `num_layers=4`. The idea is that by adding more layers, we can create a more expressive network that can understand more complex relationships.

In [9]:
# Define a new model, don't load in old weights.
deep_model = RDLModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=4,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)
deep_optimizer = torch.optim.Adam(deep_model.parameters(), lr=0.005)
deep_state_dict = training_run(
    deep_model,
    device,
    deep_optimizer,
    task,
    loader_dict,
    val_table,
    loss_fn,
    entity_table,
    epochs=10,
)
deep_model.load_state_dict(deep_state_dict)

# Evaluate on val and test set
eval_model(deep_model, loader_dict, "val", task, device, val_table)
eval_model(deep_model, loader_dict, "test", task, device, None)

100%|██████████| 23/23 [00:03<00:00,  6.41it/s]


Epoch: 01, Train loss: 0.38461807022304917, Val metrics: {'average_precision': np.float64(0.8349931032640058), 'accuracy': 0.7791519434628975, 'f1': np.float64(0.8758689175769613), 'roc_auc': np.float64(0.586702947845805)}


100%|██████████| 23/23 [00:03<00:00,  6.34it/s]


Epoch: 02, Train loss: 0.35507741827727013, Val metrics: {'average_precision': np.float64(0.8354016392187892), 'accuracy': 0.7791519434628975, 'f1': np.float64(0.8758689175769613), 'roc_auc': np.float64(0.5987120181405896)}


100%|██████████| 23/23 [00:03<00:00,  6.36it/s]


...

Epoch: 08, Train loss: 0.3110587035060378, Val metrics: {'average_precision': np.float64(0.8881105700586569), 'accuracy': 0.657243816254417, 'f1': np.float64(0.7690476190476191), 'roc_auc': np.float64(0.6688072562358276)}


100%|██████████| 23/23 [00:03<00:00,  6.29it/s]


Epoch: 09, Train loss: 0.3075667902188698, Val metrics: {'average_precision': np.float64(0.893226899686731), 'accuracy': 0.7703180212014135, 'f1': np.float64(0.8681541582150102), 'roc_auc': np.float64(0.6965079365079365)}


100%|██████████| 23/23 [00:03<00:00,  6.01it/s]


Epoch: 10, Train loss: 0.3054002355175321, Val metrics: {'average_precision': np.float64(0.8906464430139396), 'accuracy': 0.6731448763250883, 'f1': np.float64(0.7784431137724551), 'roc_auc': np.float64(0.6812154195011338)}
Best val metrics: {'average_precision': np.float64(0.8932790047558226), 'accuracy': 0.7703180212014135, 'f1': np.float64(0.8681541582150102), 'roc_auc': np.float64(0.6966167800453514)}
Best test metrics: {'average_precision': np.float64(0.8392165116414603), 'accuracy': 0.688034188034188, 'f1': np.float64(0.8120171673819743), 'roc_auc': np.float64(0.7078661006197238)}


As we see, training using $4$ layers actually makes the model perform worse over time. Thus, given the simplicity of the task and the size of dataset, it is likely that we are overfitting.

Given that using double the amount of layers leads to less optimal results, we can try the opposite strategy and halve the number of layers in the network.

In [10]:
# Define a new model, don't load in old weights.
shallow_model = RDLModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=1,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)
shallow_optimizer = torch.optim.Adam(shallow_model.parameters(), lr=0.005)
shallow_state_dict = training_run(
    shallow_model,
    device,
    shallow_optimizer,
    task,
    loader_dict,
    val_table,
    loss_fn,
    entity_table,
    epochs=10,
)
deep_model.load_state_dict(deep_state_dict)

# Evaluate on val and test set
eval_model(deep_model, loader_dict, "val", task, device, val_table)
eval_model(deep_model, loader_dict, "test", task, device, None)

100%|██████████| 23/23 [00:02<00:00, 11.06it/s]


Epoch: 01, Train loss: 0.37936397961767454, Val metrics: {'average_precision': np.float64(0.86254562502342), 'accuracy': 0.7756183745583038, 'f1': np.float64(0.8718466195761857), 'roc_auc': np.float64(0.6527528344671202)}


100%|██████████| 23/23 [00:02<00:00, 11.10it/s]


Epoch: 02, Train loss: 0.3252705187036387, Val metrics: {'average_precision': np.float64(0.8940563305132209), 'accuracy': 0.7756183745583038, 'f1': np.float64(0.871587462082912), 'roc_auc': np.float64(0.6891428571428571)}


100%|██████████| 23/23 [00:02<00:00, 11.03it/s]


...

Epoch: 08, Train loss: 0.2899213394089905, Val metrics: {'average_precision': np.float64(0.9080739212065995), 'accuracy': 0.7190812720848057, 'f1': np.float64(0.8170310701956272), 'roc_auc': np.float64(0.7196734693877551)}


100%|██████████| 23/23 [00:02<00:00, 10.82it/s]


Epoch: 09, Train loss: 0.2870603939102906, Val metrics: {'average_precision': np.float64(0.9035548667517451), 'accuracy': 0.696113074204947, 'f1': np.float64(0.7957244655581948), 'roc_auc': np.float64(0.712689342403628)}


100%|██████████| 23/23 [00:02<00:00, 10.92it/s]


Epoch: 10, Train loss: 0.2900424680871974, Val metrics: {'average_precision': np.float64(0.9025229009251505), 'accuracy': 0.7703180212014135, 'f1': np.float64(0.8694779116465864), 'roc_auc': np.float64(0.7153378684807257)}
Best val metrics: {'average_precision': np.float64(0.8932453499605095), 'accuracy': 0.7703180212014135, 'f1': np.float64(0.8681541582150102), 'roc_auc': np.float64(0.6965260770975057)}
Best test metrics: {'average_precision': np.float64(0.8392455582033577), 'accuracy': 0.688034188034188, 'f1': np.float64(0.8120171673819743), 'roc_auc': np.float64(0.7079539354901674)}


Interestingly enough, we see that even with half the number of layers, we do just about the same as with double the number of layers. Once again, this might be more task-specific as opposed to a general conclusion about GNNs and the RDL pipeline.

### Challenge
Does this trend necessarily hold on larger and more diverse datasets? Depending on your compute availability, try out using different datasets, like `rel-amazon`, as well as across different types of tasks!